In [ ]:
import mne ## MNE-Python for analysing data
%matplotlib widget 
from os import chdir
from os.path import join
import matplotlib.pyplot as plt ## for basic plotting
import matplotlib as mpl ## for setting default parameters
import pandas as pd
import numpy as np

In [ ]:
#%% SET DEFAULT PLOTTING PARAMETERS

mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['font.size'] = 12
mpl.rcParams['font.weight'] = 'bold'
mpl.rcParams['lines.linewidth'] = 3
plt.ion()

In [ ]:
MEG_path = '/work/MEG_data/workshop_data/0168/20250924_000000' ## you can pick your study group member
subjects_dir = '/work/freesurfer'
behaviour_path = '/work/MEG_data/workshop_data/behavioural_logs'

In [ ]:
raw = mne.io.read_raw_fif(join(MEG_path, 'workshop_2025_raw.fif'), preload=True) 
fig = raw.plot() ## you will note this is completely saturated (all blue) do the PSD and find out why

In [ ]:
#copy_bandpass = raw.copy()
#copy_bandpass.filter(h_freq=40,   l_freq=1); ## bandpass fillter of 1-40 Hz

raw.filter(h_freq=40, l_freq=1); ## this method ".filter()" modifies in the object in place

In [ ]:
## what does it look after choosing an appropriate filter?
raw.compute_psd().plot()

In [ ]:
#%% FIND EVENTS

events = mne.find_events(raw, min_duration=0.002)

events_filtered = events[(events[:, 2] == 1) | (events[:, 2] == 3)]
pas_events = events_filtered.copy()   # keeps shape (n_events, 3)
behaviour = pd.read_csv(join(behaviour_path, '0168_2025_09_24_135504_experiment_data.csv'), index_col=False)
pas_events[:, 2] = behaviour['subjective_response'] + 20

In [ ]:
# Epochs
event_id = {'pas1': 21, 'pas2': 22, 'pas3': 23, 'pas4': 24}
epochs = mne.Epochs(raw, events=pas_events, event_id=event_id, tmin=-0.2, tmax=0.55, baseline=(None,0))

In [ ]:
## CREATING AVERAGES AND DIFFERENCE WAVES

### NEW CODE

evokeds = list()
for event in epochs.event_id:
    evokeds.append(epochs[event].average())

## this might help you along
evoked_diffs = list()
diffs = ['pas1', 'pas2', 'pas3','pas4']
for diff in diffs:
    evoked_diff = evokeds[0].copy() # create a copy
    evoked_diff.data -= evokeds[1].data # modify the data in place
    evoked_diff.comment = diff

In [ ]:
mne.viz.plot_evoked(evoked_diff)

In [ ]:
## GET FORWARD MODEL

### NEW CODE

info = epochs.info
trans = join(MEG_path, 'workshop_2025-trans.fif') 
src = join(subjects_dir, '0168', 'bem', '0168' + '-oct-6-src.fif')
bem = join(subjects_dir, '0168', 'bem', '0168' + '-5120-bem-sol.fif')

fwd = mne.make_forward_solution(
                            info, trans,
                            src, bem)